This is a replication of a blog post done on the Gray-Scott equations. 

In [1]:
import matplotlib.pyplot as plt
import numpy as np

# I'm using seaborn for it's fantastic default styles
#import seaborn as sns
#sns.set_style("whitegrid")

## This appears to be specific to the jupyter notebook system. 
#matplotlib inline
#load_ext autoreload
#autoreload 2

from tutils import BaseStateSystem

def laplacian1D(a, dx):
    return (
        - 2 * a
        + np.roll(a,1,axis=0) 
        + np.roll(a,-1,axis=0)
    ) / (dx ** 2)

def laplacian2D(a, dx):
    return (
        - 4 * a
        + np.roll(a,1,axis=0) 
        + np.roll(a,-1,axis=0)
        + np.roll(a,+1,axis=1)
        + np.roll(a,-1,axis=1)
    ) / (dx ** 2)

def random_initialiser(shape):
    return(
        np.random.normal(loc=0, scale=0.05, size=shape),
        np.random.normal(loc=0, scale=0.05, size=shape)
    )

Da, Db, alpha, beta = 1, 20.3, -0.005, 10

eq_pt = np.cbrt(alpha)

def pert_a(x): return np.cos(14*np.pi*x/100)
def pert_b(x): return np.cos(14*np.pi*x/100)

# Initialisation near eq_pt with some perturbation
f1 = lambda x : eq_pt*(1 + pert_a(x))
f2 = lambda x : eq_pt*(1 + pert_b(x))

def function_initialiser(shape,f1,f2):
    space = range(shape)
    return(
        np.fromiter(map(f1,space),float),
        np.fromiter(map(f2,space),float)
    )

class OneDimensionalRDEquations(BaseStateSystem):
    def __init__(self, Da, Db, Ra, Rb,
                 initialiser=random_initialiser,
                 width=1000, dx=1, 
                 dt=0.1, steps=1):
        
        self.Da = Da
        self.Db = Db
        self.Ra = Ra
        self.Rb = Rb
        
        self.initialiser = initialiser
        self.width = width
        self.dx = dx
        self.dt = dt
        self.steps = steps
        
        
    def initialise(self):
        self.t = 0
#        self.a, self.b = random_initialiser(self.width)
        self.a, self.b = function_initialiser(self.width,f1,f2)
        
    def update(self):
        for _ in range(self.steps):
            self.t += self.dt
            self._update()

    def _update(self):
        
        # unpack so we don't have to keep writing "self"
        a,b,Da,Db,Ra,Rb,dt,dx = (
            self.a, self.b,
            self.Da, self.Db,
            self.Ra, self.Rb,
            self.dt, self.dx
        )
        
        La = laplacian1D(a, dx)
        Lb = laplacian1D(b, dx)
        
        delta_a = dt * (Da * La + Ra(a,b))
        delta_b = dt * (Db * Lb + Rb(a,b))
        
        self.a += delta_a
        self.b += delta_b
        
    def draw(self, ax):
        ax.clear()
        ax.plot(self.a, color="r", label="A")
        ax.plot(self.b, color="b", label="B")
        ax.legend()
        ax.set_ylim(-1,1)
        ax.set_title("t = {:.2f}".format(self.t))
    

def Ra(a,b): return a - a ** 3 - b + alpha
def Rb(a,b): return (a - b) * beta

width = 100
dx = 1
dt = 0.001

Equation = OneDimensionalRDEquations(
    Da, Db, Ra, Rb, 
    width=width, dx=dx, dt=dt, 
    steps=100
)

#Equation.plot_time_evolution("1dRD_check_2.gif", n_steps=100)
Equation.plot_evolution_outcome("1dRD.png", n_steps=100)
